In [1]:
from unsloth import FastLanguageModel
import torch

# Configuration
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load the selected model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

/home/ubuntu/miniconda3/envs/unsloth/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.536 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [15]:
import pandas as pd
from pprint import pprint

list_ds1 =[
{"instruction":"Which company is the Acquirer.", "input":"Peak Performance Corporation to Acquire Power Forward Corporation by 28/Jul/2022", "output":"Peak Performance Corporation"},
{"instruction":"Which company is the Acquirer.", "input":"Prime Solutions Group Acquires Dynamic Solutions Inc in a Strategic Merger", "output":"Prime Solutions Group"},
{"instruction":"Which company is the Acquirer.", "input":"Wonderful Products Inc, a leading company in the consumer goods industry, has successfully completed its acquisition of Noble Solutions Group, a prominent provider of innovative solutions.", "output":"Wonderful Products Inc"},

{"instruction":"Which company is the Acquirer.", "input":"ABC Corporation has completed acquisition of 91% stake in French IT Consulting Company, Rene Consulting", "output":"ABC Corporation"},
{"instruction":"Which company is the Acquirer.", "input":"JBL Technologies has completed acquisition of 51% stake in California Based Defence Company, MaryLand", "output":"JBL Technologies"},

 {"instruction":"Which company is the Acquirer.", "input":"New York, NY - Empire Innovations Inc, a leading technology conglomerate, announced today its plans to acquire Unique Ventures LLC, a prominent venture capital firm specializing in disruptive startups.", "output":"Empire Innovations Inc"},
{"instruction":"Which company is the Acquirer.", "input":"Summit Solutions Inc, a leading technology solutions provider, has announced its plans to acquire Accelerate Innovations Group, a prominent innovation and consulting firm.", "output":"Summit Solutions Inc"}
]

list_ds=[

{"instruction":"Which company is the Acquirer.", "input":"Tech Giants Inc. to Purchase Majority Stake in Green Energy Corp by 31/Dec/2023", "output":"Tech Giants Inc."},
{"instruction":"Which company is the Acquirer.", "input":"Pinnacle Holdings Group Completes Acquisition of Nova Spire Inc, a Leading Fintech Company", "output":"Pinnacle Holdings Group"},
{"instruction":"Which company is the Acquirer.", "input":"Global Horizon Ventures, a leading private equity firm, has acquired a significant stake in Zenith Industries Ltd., a prominent player in the manufacturing sector.", "output":"Global Horizon Ventures"},


{"instruction":"Which company is the Acquirer.", "input":"Vertex Pharmaceuticals Inc. Announces Plans to Acquire 60% Stake in Boston-Based Biotech Firm, Helix Therapeutics", "output":"Vertex Pharmaceuticals Inc."},
{"instruction":"Which company is the Acquirer.", "input":"SynergyCorp Ltd. Completes Acquisition of UK-Based FinTech Company, Pulse Payments plc", "output":"SynergyCorp Ltd."},


{"instruction":"Which company is the Acquirer.", "input":"Denver, CO - Altitude Capital Partners, a leading investment firm, announced today its plans to acquire ClearView Ventures LLC, a prominent venture capital firm specializing in emerging technologies.", "output":"Altitude Capital Partners"},
{"instruction":"Which company is the Acquirer.", "input":"Apex Systems Inc., a leading technology solutions provider, has announced its plans to acquire Catalyst Consulting Group, a prominent innovation and strategy consulting firm.", "output":"Apex Systems Inc."},
{"instruction":"Which company is the Acquirer.", "input":"Hong Kong - Pacific Rim Holdings Ltd. Completes Acquisition of 80% Stake in Shanghai-Based E-Commerce Company, DragonFly Technologies", "output":"Pacific Rim Holdings Ltd."}

]



In [16]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [17]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

import datasets
import pandas as pd
# Create a dataset from the array of dictionaries

df = pd.DataFrame(list_ds)
dataset = datasets.Dataset.from_pandas(df)

# Print the dataset
print(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 8
})


Map: 100%|██████████| 8/8 [00:00<00:00, 2100.04 examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 8
})


In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Number of processes to use for data preprocessing
    packing = False, # Whether to pack multiple sequences into one batch to increase training efficiency
    args = TrainingArguments(
        per_device_train_batch_size = 2, #The batch size per device
        gradient_accumulation_steps = 4, #Number of gradient accumulation steps, which allows for effectively larger batch sizes
        warmup_steps = 5, #Number of steps to perform linear learning rate warmup
        max_steps = 60, #Total number of training steps
        learning_rate = 2e-5,#The learning rate for the optimizer
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()


Map (num_proc=2):   0%|          | 0/8 [00:00<?, ? examples/s]

Map (num_proc=2): 100%|██████████| 8/8 [00:00<00:00,  9.46 examples/s]
/home/ubuntu/miniconda3/envs/unsloth/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 8 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.685300
2,1.547400
3,1.524900
4,1.629200
5,1.414100
6,1.338200
7,1.375400
8,1.254200
9,1.252300
10,1.171900


In [19]:
mna_news_input = """London, UK - Oakwood Private Equity LLP, a leading investment firm, announced today its plans to acquire 75% stake in Amsterdam-Based Digital Marketing Agency, Bloom Media BV"""

mna_news_instruction1 = """Which company is the Acquirer"""
mna_news_instruction2 = """what percentage of stake is getting acquire by Oakwood Private Equity LLP """
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        mna_news_input, # instruction
        mna_news_instruction1, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
generated_text = tokenizer.batch_decode(outputs)
generated_text

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nLondon, UK - Oakwood Private Equity LLP, a leading investment firm, announced today its plans to acquire 75% stake in Amsterdam-Based Digital Marketing Agency, Bloom Media BV\n\n### Input:\nWhich company is the Acquirer\n\n### Response:\nOakwood Private Equity LLP<|end_of_text|>']

In [20]:
response = generated_text[0].split("\n\n### Response:\n")[1].split("\n\n### Explanation:\n")[0]

In [21]:
response

'Oakwood Private Equity LLP<|end_of_text|>'

In [22]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...")
# tokenizer.push_to_hub("your_name/lora_model", token = "...")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [23]:
# if False:
#     from unsloth import FastLanguageModel
#     model, tokenizer = FastLanguageModel.from_pretrained(
#         model_name = "lora_model",
#         max_seq_length = max_seq_length,
#         dtype = dtype,
#         load_in_4bit = load_in_4bit,
#     )
#     FastLanguageModel.for_inference(model)

from unsloth import FastLanguageModel
model_l, tokenizer_l = FastLanguageModel.from_pretrained(
    model_name = "lora_model",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model_l)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.536 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [24]:
inputs = tokenizer_l(
[
    alpaca_prompt.format(
        mna_news_input, # instruction
        mna_news_instruction2, # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model_l.generate(**inputs, max_new_tokens = 64, use_cache = True)
generated_text = tokenizer.batch_decode(outputs)

In [25]:
response = generated_text[0].split("\n\n### Response:\n")[1].split("\n\n### Explanation:\n")[0]

In [26]:
response

'Oakwood Private Equity LLP is acquiring 75% stake in Amsterdam-Based Digital Marketing Agency, Bloom Media BV<|end_of_text|>'